# Start spark application (spark session)

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1649343105521_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load ticket data from the S3 bucket

In [2]:
s3_path = "s3a://upgrad-data/Parking_Violation_Tickets.csv"
tickets = spark.read.csv(s3_path, header = True, inferSchema = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check schema of the dataframe

In [3]:
tickets.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

# Import all SQL functions and create SQL view

In [4]:
from pyspark.sql.functions import *
tickets.createOrReplaceTempView("tickets_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check for duplicate entries

In [5]:
spark.sql("\
            SELECT `Summons Number`, COUNT(`Summons Number`) AS `Number of duplicate occurance of summons number`\
            FROM tickets_view\
            GROUP BY `Summons Number`\
            HAVING COUNT(`Summons Number`) > 1\
        ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------------------------------------------+
|Summons Number|Number of duplicate occurance of summons number|
+--------------+-----------------------------------------------+
+--------------+-----------------------------------------------+

# Total number of tickets

In [6]:
tickets.select(tickets["Summons Number"]).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10803028

# Number of unique states from where cars got parking ticket

In [7]:
spark.sql("\
            SELECT COUNT(`Registration State`) as `Number of unique states`\
            FROM\
            (\
                SELECT `Registration State`\
                FROM tickets_view\
                GROUP BY `Registration State`\
            ) AS SUBQUERY\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|Number of unique states|
+-----------------------+
|                     67|
+-----------------------+

# State having the maximum tickets

In [8]:
spark.sql("\
            SELECT `Registration State`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Registration State`\
            ORDER BY `Number of Tickets` DESC\
            LIMIT 1\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+
|Registration State|Number of tickets|
+------------------+-----------------+
|                NY|          8481061|
+------------------+-----------------+

# Replace the numeric entry '99' with the state having maximum tickets

In [9]:
tickets_updated = tickets.withColumn("Registration State", when(tickets["Registration State"] == '99', 'NY').otherwise(tickets["Registration State"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create SQL view of the updated dataframe

In [10]:
tickets_updated.createOrReplaceTempView("tickets_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Number of unique states from where cars got parking ticket


In [11]:
spark.sql("\
            SELECT COUNT(`Registration State`) as `Number of unique states`\
            FROM\
            (\
                SELECT `Registration State`\
                FROM tickets_view\
                GROUP BY `Registration State`\
            ) AS SUBQUERY\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|Number of unique states|
+-----------------------+
|                     66|
+-----------------------+

# Top twenty states with the most number of tickets

In [12]:
spark.sql("\
            SELECT `Registration State`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Registration State`\
            ORDER BY `Number of Tickets` DESC\
            LIMIT 20\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+
|Registration State|Number of tickets|
+------------------+-----------------+
|                NY|          8517686|
|                NJ|           925965|
|                PA|           285419|
|                FL|           144556|
|                CT|           141088|
|                MA|            85547|
|                IN|            80749|
|                VA|            72626|
|                MD|            61800|
|                NC|            55806|
|                IL|            37329|
|                GA|            36852|
|                TX|            36516|
|                AZ|            26426|
|                OH|            25302|
|                CA|            24260|
|                SC|            21836|
|                ME|            21574|
|                MN|            18227|
|                OK|            18165|
+------------------+-----------------+

# Frequency of top five violation codes

In [13]:
spark.sql("\
            SELECT `Violation Code`, COUNT(`Summons Number`) as `Number of violations`\
            FROM tickets_view\
            GROUP BY `Violation Code`\
            ORDER BY `Number of violations` DESC\
            LIMIT 5\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------------+
|Violation Code|Number of violations|
+--------------+--------------------+
|            21|             1528588|
|            36|             1400614|
|            38|             1062304|
|            14|              893498|
|            20|              618593|
+--------------+--------------------+

# Frequency of top five vehicle body type

In [14]:
spark.sql("\
            SELECT `Vehicle Body Type`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Vehicle Body Type`\
            ORDER BY `Number of tickets` DESC\
            LIMIT 5\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+
|Vehicle Body Type|Number of tickets|
+-----------------+-----------------+
|             SUBN|          3719802|
|             4DSD|          3082020|
|              VAN|          1411970|
|             DELV|           687330|
|              SDN|           438191|
+-----------------+-----------------+

# Frequency of top five vehicle make

In [15]:
spark.sql("\
            SELECT `Vehicle Make`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Vehicle Make`\
            ORDER BY `Number of tickets` DESC\
            LIMIT 5\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------+
|Vehicle Make|Number of tickets|
+------------+-----------------+
|        FORD|          1280958|
|       TOYOT|          1211451|
|       HONDA|          1079238|
|       NISSA|           918590|
|       CHEVR|           714655|
+------------+-----------------+

# Divide the data into seasons

# Assumptions

|Season|Months|Months in numbers|
|------|------|------|
|Summer|March, April, May|3,4,5|
|Monsoon|June, July, August|6,7,8|
|Autumn|September, October, November|9,10,11|
|Winter|December, January, February|12,1,2|

# Cast string type as date type

In [16]:
tickets_date_updated = tickets_updated.withColumn("Issue Date", to_date(col("Issue Date"), "MM/dd/yyyy"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create SQL view of the updated dataframe

In [17]:
tickets_date_updated.createOrReplaceTempView("tickets_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create dataframe segregating the data as per seasons defined in above assumption and keep only the required colums for further analysis

In [18]:
tickets_seasonal = spark.sql("\
                                SELECT `Summons Number`, `Issue Date`, `Violation Code`,\
                                CASE\
                                    WHEN MONTH(`Issue Date`) IN (3,4,5) THEN 'Summer'\
                                    WHEN MONTH(`Issue Date`) IN (6,7,8) THEN 'Monsoon'\
                                    WHEN MONTH(`Issue Date`) IN (9,10,11) THEN 'Autumn'\
                                    WHEN MONTH(`Issue Date`) IN (12,1,2) THEN 'Winter'\
                                    ELSE 'Invalid'\
                                END AS `Season`\
                                FROM tickets_view\
                            ")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create SQL view of the dataframe

In [19]:
tickets_seasonal.createOrReplaceTempView("tickets_seasonal_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Seasonal frequency of tickets

In [20]:
spark.sql("\
            SELECT `Season`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_seasonal_view\
            GROUP BY `Season`\
            ORDER BY `Number of tickets` DESC\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
| Season|Number of tickets|
+-------+-----------------+
| Summer|          2880687|
| Autumn|          2830802|
|Monsoon|          2606208|
| Winter|          2485331|
+-------+-----------------+

# Three most common violations for each season

### Summer

In [21]:
spark.sql("\
            SELECT `Violation Code`, COUNT(`Summons Number`) AS `Frequency`\
            FROM `tickets_seasonal_view`\
            WHERE `Season` = 'Summer'\
            GROUP BY `Violation Code`\
            ORDER BY `Frequency` DESC\
            LIMIT 3\
          ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   402807|
|            36|   344834|
|            38|   271192|
+--------------+---------+

### Monsoon

In [22]:
spark.sql("\
            SELECT `Violation Code`, COUNT(`Summons Number`) AS `Frequency`\
            FROM `tickets_seasonal_view`\
            WHERE `Season` = 'Monsoon'\
            GROUP BY `Violation Code`\
            ORDER BY `Frequency` DESC\
            LIMIT 3\
          ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   405961|
|            38|   247561|
|            36|   240396|
+--------------+---------+

### Autumn

In [23]:
spark.sql("\
            SELECT `Violation Code`, COUNT(`Summons Number`) AS `Frequency`\
            FROM `tickets_seasonal_view`\
            WHERE `Season` = 'Autumn'\
            GROUP BY `Violation Code`\
            ORDER BY `Frequency` DESC\
            LIMIT 3\
          ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            36|   456046|
|            21|   357479|
|            38|   283828|
+--------------+---------+

### Winter

In [24]:
spark.sql("\
            SELECT `Violation Code`, COUNT(`Summons Number`) AS `Frequency`\
            FROM `tickets_seasonal_view`\
            WHERE `Season` = 'Winter'\
            GROUP BY `Violation Code`\
            ORDER BY `Frequency` DESC\
            LIMIT 3\
          ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   362341|
|            36|   359338|
|            38|   259723|
+--------------+---------+

# Total occurance of three most common violation codes

In [25]:
spark.sql("\
            SELECT SUM(`Number of tickets`) AS `Total number of tickets for three most common violation codes`\
            FROM\
            (\
                SELECT `Violation Code`, COUNT(`Summons Number`) AS `Number of Tickets`\
                FROM tickets_view\
                GROUP BY `Violation Code`\
                ORDER BY `Number of tickets` DESC\
                LIMIT 3\
            ) AS SUBQUERY\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------+
|Total number of tickets for three most common violation codes|
+-------------------------------------------------------------+
|                                                      3991506|
+-------------------------------------------------------------+

# Assumptions

|Rank|Violation Code|Manhattan 96th St. & below|All other areas|Average Fine|
|------|------|------|------|------|
|1|21|65|65|65|
|2|36|50|50|50|
|3|38|65|35|50|


### All the calculations related to revenue are based upon revenue generated by collecting fines for above three violation codes only

# Create dataframe containing column of fines along with columns required for revenue analysis

In [26]:
tickets_revenue = spark.sql("\
                                SELECT *\
                                FROM\
                                (\
                                    SELECT `Summons Number`, `Violation Code`, `Registration State`,\
                                    CASE\
                                        WHEN `Violation Code` = '21' THEN '65'\
                                        WHEN `Violation Code` = '36' THEN '50'\
                                        WHEN `Violation Code` = '38' THEN '50'\
                                        ELSE '0'\
                                    END AS `Average fine`\
                                    FROM tickets_view\
                                ) AS SUBQUERY\
                                WHERE `Average fine` <> '0'\
                            ")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create SQL view of the dataframe

In [27]:
tickets_revenue.createOrReplaceTempView("tickets_revenue_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Total amount of revenue collected for each violation

In [28]:
revenue = spark.sql("\
                        SELECT `Violation Code`, SUM(`Average fine`) AS `Revenue collected`\
                        FROM tickets_revenue_view\
                        GROUP BY `Violation Code`\
                        ORDER BY `Revenue collected` DESC\
                    ")
revenue.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------------+
|Violation Code|Revenue collected|
+--------------+-----------------+
|            21|       9.935822E7|
|            36|        7.00307E7|
|            38|        5.31152E7|
+--------------+-----------------+

# Violation code with highest total collection

In [29]:
revenue.select("Violation Code").show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|Violation Code|
+--------------+
|            21|
+--------------+
only showing top 1 row

# Top three states that have highest ticket revenue

In [30]:
spark.sql("\
            SELECT `Registration State`, SUM(`Average fine`) AS `Revenue collected`\
            FROM tickets_revenue_view\
            GROUP BY `Registration State`\
            ORDER BY `Revenue collected` DESC\
            LIMIT 3\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+
|Registration State|Revenue collected|
+------------------+-----------------+
|                NY|      1.7707888E8|
|                NJ|       1.475377E7|
|                PA|        6944560.0|
+------------------+-----------------+

# Inference from the analysis

1. State with highest number of tickets is **New York**

2. The most common violation code is **21**, *Street Cleaning: No parking where parking is not allowed by sign, street marking or traffic control device.*

3. Most common violations are by vehicle body type **SUBN** *Sub-urban*

4. Most common violations are by vehicle make **Ford**

5. Most violations occur in **Summer**

6. Least violations occur in **Winter**

7. In autum, the most common violation code is **36**, *Exceeding the posted speed limit in or near a designated school zone.*